In [1]:
import os 
import openai 
import sys 

In [2]:
from dotenv import load_dotenv
load_dotenv()

# Get API key and model name from environment
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_model = os.getenv("OPENAI_MODEL")

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
# Initialize the LLM with a valid model name
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # Use a valid OpenAI model name
    temperature=0
)


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = r'C:\Users\karti\Desktop\Project\openai_learning\vector_database\Learning'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory = persist_directory, embedding_function=embedding)

C:\Users\karti\AppData\Local\Temp\ipykernel_13504\800757705.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
C:\Users\karti\AppData\Local\Temp\ipykernel_13504\800757705.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory = persist_directory, embedding_function=embedding)


In [6]:
print(vectordb._collection.count())

165


In [7]:
question = "what are the major thing discuss in the pdf"
docs = vectordb.similarity_search(question , k = 5)

In [8]:
len(docs )

5

In [9]:
for i in range(len(docs)):
    print(docs[i])

page_content='Vision–ECCV 2014, pages 299–314. Springer, 2014. 7
[8] D. Erhan, C. Szegedy, A. Toshev, and D. Anguelov. Scalable
object detection using deep neural networks. In Computer
Vision and Pattern Recognition (CVPR), 2014 IEEE Confer-
ence on, pages 2155–2162. IEEE, 2014. 5, 6
[9] M. Everingham, S. M. A. Eslami, L. Van Gool, C. K. I.
Williams, J. Winn, and A. Zisserman. The pascal visual ob-
ject classes challenge: A retrospective.International Journal
of Computer Vision, 111(1):98–136, Jan. 2015. 2
[10] P. F. Felzenszwalb, R. B. Girshick, D. McAllester, and D. Ra-
manan. Object detection with discriminatively trained part
based models. IEEE Transactions on Pattern Analysis and
Machine Intelligence, 32(9):1627–1645, 2010. 1, 4
[11] S. Gidaris and N. Komodakis. Object detection via a multi-
region & semantic segmentation-aware CNN model. CoRR,
abs/1505.01749, 2015. 7
[12] S. Ginosar, D. Haas, T. Brown, and J. Malik. Detecting peo-
ple in cubist art. InComputer Vision-ECCV 2014 Wo

In [10]:
from langchain.chains import RetrievalQA

In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever()
)

In [12]:
question = "what is the pdf about"

In [13]:
docs = qa_chain.retriever.get_relevant_documents(question)
print("Retrieved Documents:", docs)


C:\Users\karti\AppData\Local\Temp\ipykernel_13504\3836497603.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = qa_chain.retriever.get_relevant_documents(question)


Retrieved Documents: [Document(metadata={'page': 0, 'page_label': '1', 'source': 'C:\\Users\\karti\\Desktop\\Project\\openai_learning\\docs\\yolo.pdf'}, page_content='means we can process streaming video in real-time with\nless than 25 milliseconds of latency. Furthermore, YOLO\nachieves more than twice the mean average precision of\nother real-time systems. For a demo of our system running\nin real-time on a webcam please see our project webpage:\nhttp://pjreddie.com/yolo/.\nSecond, YOLO reasons globally about the image when\n1\narXiv:1506.02640v5  [cs.CV]  9 May 2016'), Document(metadata={'page': 0, 'page_label': '1', 'source': 'C:\\Users\\karti\\Desktop\\Project\\openai_learning\\docs\\yolo.pdf'}, page_content='means we can process streaming video in real-time with\nless than 25 milliseconds of latency. Furthermore, YOLO\nachieves more than twice the mean average precision of\nother real-time systems. For a demo of our system running\nin real-time on a webcam please see our project 

In [14]:
try:
    result = qa_chain({"query": question})
except Exception as e:
    print("Error:", str(e))

C:\Users\karti\AppData\Local\Temp\ipykernel_13504\1247687008.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [15]:
result

{'query': 'what is the pdf about',
 'result': "I don't have enough context to determine what specific PDF you are referring to. If you can provide more details or a specific title, I may be able to help you better."}

In [16]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # need to specify parameter name
    retriever=vectordb.as_retriever(),
    return_source_documents=True,  # correct parameter name
    chain_type="stuff",  # specify chain type
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}  # correct dictionary syntax
)


In [18]:
# Example usage
query = "what is simense network" 
result = qa_chain({"query": query})

In [19]:
result["result"]

'Siamese networks are a type of neural network architecture that are sensitive to calibration and context when determining similarity between objects or individuals. They are often used for tasks such as image recognition and classification. Thanks for asking!'

In [20]:
result["source_documents"]

[Document(metadata={'page': 0, 'page_label': '1', 'source': 'C:\\Users\\karti\\Desktop\\Project\\openai_learning\\docs\\Triplet_loss_original_paper.pdf'}, page_content='Siamese networks are also sensitive to calibration in the sense that the notion of similarity vs dis-\nsimilarity requires context. For example, a person might be deemed similar to another person when\na dataset of random objects is provided, but might be deemed dissimilar with respect to the same\nother person when we wish to distinguish between two individuals in a set of individuals only. In\nour model, such a calibration is not required. In fact, in our experiments here, we have experienced\nhands on the difﬁculty in using Siamese networks.\nWe follow a similar task to that of Chechik et al. (2010). For a set of samples P and a chosen rough\nsimilarity measure r(x, x′) given through a training oracle (e.g how close are two images of objects\n∗The author acknowledges the generous support of ISF grant number 1271/13\n

In [21]:
question = "is math important"

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # need to specify parameter name
    retriever=vectordb.as_retriever(),
    return_source_documents=True,  # correct parameter name
    chain_type="map_reduce",  # specify chain type
)


In [23]:
result = qa_chain({"query" : query})

In [24]:
result['result']

'A Siamese network is a type of neural network architecture that contains two identical subnetworks which are joined at the output layer. This architecture is commonly used for tasks involving similarity or distance measurement, such as image recognition, signature verification, and more. The network is trained to learn how to differentiate between similar and dissimilar inputs by minimizing the distance between similar pairs and maximizing the distance between dissimilar pairs in the output space.'

In [25]:
import panel as pn 
pn.extension()

In [26]:
from langchain.memory import ConversationBufferMemory

In [27]:
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages= True 
)

C:\Users\karti\AppData\Local\Temp\ipykernel_13504\1160254780.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [28]:
from langchain.chains import ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # need to specify parameter name
    retriever=vectordb.as_retriever(),
    memory = memory
)


In [29]:
question = "what are the pdf about"
result = qa_chain({'question' : question})

In [30]:
result

{'question': 'what are the pdf about',
 'chat_history': [HumanMessage(content='what are the pdf about', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know.", additional_kwargs={}, response_metadata={})],
 'answer': "I don't know."}

In [31]:
result['answer']

"I don't know."

In [32]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [33]:
import param

In [40]:
# Function to load the database
def load_db(file, chain_type, k):
    # Load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    
    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    
    # Define embedding
    embeddings = OpenAIEmbeddings()
    
    # Create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    
    # Define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    
    # Create a chatbot chain
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

# ChatBot class
class ChatBot(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query = param.String("")
    db_response = param.List([])
    
    def __init__(self, **params):
        super().__init__(**params)
        self.panels = []
        self.loaded_file = r"C:\Users\karti\Desktop\Project\openai_learning\docs\Triplet_loss_original_paper.pdf"
        self.qa = load_db(self.loaded_file, "stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        
        with open("temp.pdf", "wb") as f:
            f.write(file_input.value)
        
        self.loaded_file = file_input.filename
        button_load.button_type = "primary"
        self.qa = load_db("temp.pdf", "stuff", 4)
        button_load.button_type = "success"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
    
    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.append((query, result["answer"]))
        self.db_query = result.get("generated_question", "")
        self.db_response = result.get("source_documents", [])
        self.answer = result['answer']
        
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, styles={'background-color': '#F6F6F6'}))
        ])
        
        return pn.WidgetBox(*self.panels, scroll=True)

    @param.depends('db_query')
    def get_lquest(self):
        return pn.Column(
            pn.pane.Markdown("Last query to DB:" if self.db_query else "No DB accesses yet", styles={'background-color': '#F6F6F6'}),
            pn.pane.Str(self.db_query or "No DB accesses so far")
        )
    
    @param.depends('db_response')
    def get_sources(self):
        if not self.db_response:
            return pn.pane.Markdown("No sources yet")
        
        return pn.WidgetBox(
            pn.pane.Markdown("Result of DB lookup:", styles={'background-color': '#F6F6F6'}),
            *[pn.pane.Str(doc) for doc in self.db_response],
            width=600, scroll=True
        )

    @param.depends('chat_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.pane.Str("No History Yet"), width=600, scroll=True)
        
        return pn.WidgetBox(
            pn.pane.Markdown("Current Chat History", styles={'background-color': '#F6F6F6'}),
            *[pn.pane.Str(str(exchange)) for exchange in self.chat_history],
            width=600, scroll=True
        )
    
    def clr_history(self, event=None):
        self.chat_history = []
        self.panels = []

# Initialize components
chatbot = ChatBot()
file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(chatbot.clr_history)
inp = pn.widgets.TextInput(placeholder='Enter text here…')

# Bind button and conversation
bound_button_load = pn.bind(chatbot.call_load_db, button_load.param.clicks)
conversation = pn.bind(chatbot.convchain, inp)

# Create tabs
tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=300),
    pn.layout.Divider(),
)

tab2 = pn.Column(
    pn.panel(chatbot.get_lquest),
    pn.layout.Divider(),
    pn.panel(chatbot.get_sources),
)

tab3 = pn.Column(
    pn.panel(chatbot.get_chats),
    pn.layout.Divider(),
)

tab4 = pn.Column(
    pn.Row(file_input, button_load, bound_button_load),
    pn.Row(button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic")),
    pn.layout.Divider(),
)

# Create dashboard
dashbaord = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(
        ('Conversation', tab1),
        ('Database', tab2),
        ('Chat History', tab3),
        ('Configure', tab4)
    )
)

dashbaord.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'357d5d31-b7e7-4eec-aef1-89310b874cca': {'version…